<a href="https://colab.research.google.com/github/firarru/Text-Mining/blob/main/Sistem_rekomendasi_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
# pip install --upgrade pandas

# **Loading Data**

In [4]:
# pip install kagglehub

In [5]:
import kagglehub

path = kagglehub.dataset_download("artermiloff/steam-games-dataset")
print("Path to dataset files:", path)

import os

# Cek file yang tersedia dalam dataset
files = os.listdir(path)
print("Files in dataset:", files)

# Sesuaikan dengan file yang ada - corrected file name
dataset_path = os.path.join(path, "games_march2025_cleaned.csv")
df = pd.read_csv(dataset_path)

# Tampilkan 5 baris pertama
df.head(10)

100%|██████████| 418M/418M [00:05<00:00, 83.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/artermiloff/steam-games-dataset/versions/2
Files in dataset: ['games_may2024_cleaned.csv', 'games_may2024_full.csv', 'games_march2025_cleaned.csv', 'games_march2025_full.csv']


,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608
5,440,Team Fortress 2,2007-10-10,0,0.00,1,'The most fun you can have online' - PC Gamer ...,'The most fun you can have online' - PC Gamer ...,Nine distinct classes provide a broad range of...,NaN,...,0,0,0,0,50817,"{'Free to Play': 62868, 'Hero Shooter': 61020,...",89,1146642,93,8172
6,105600,Terraria,2011-05-16,0,9.99,2,"Dig, Fight, Explore, Build: The very world is ...","Dig, Fight, Explore, Build: The very world is ...","Dig, fight, explore, build! Nothing is impossi...",NaN,...,0,0,0,0,30516,"{'Open World Survival Craft': 16365, 'Sandbox'...",97,1102434,96,8860
7,252490,Rust,2018-02-08,0,39.99,4,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,The only aim in Rust is to survive. Everything...,"“Rust is one of the cruelest games on Steam, a...",...,836,3675,231,0,200902,"{'Survival': 18592, 'Crafting': 11822, 'Multip...",87,993856,87,14960
8,4000,Garry's Mod,2006-11-29,0,5.99,0,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,Garry's Mod is a physics sandbox. There aren't...,“It brings back the silly glee of unbridled ex...,...,100,1173,136,40,32384,"{'Sandbox': 18706, 'Moddable': 14479, 'Multipl...",96,985010,96,8277
9,1172470,Apex Legends™,2020-11-04,0,0.00,0,Apex Legends: Takeover About the Game Conquer ...,"Conquer with character in Apex Legends, a free...","Apex Legends is the award-winning, free-to-pla...",“The champion of Battle Royales.” 9/10 – GameS...,...,548,667,211,0,151844,"{'Free to Play': 2170, 'Battle Royale': 1483, ...",67,983230,53,7054


In [6]:
df = df[['name', 'release_date', 'detailed_description', 'tags']]

In [7]:
df['release_year'] = pd.to_datetime(df['release_date']).dt.year

<ipython-input-7-4c1c43b0180d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = pd.to_datetime(df['release_date']).dt.year


In [8]:
df.isnull().sum()/100

,0
name,0.00
release_date,0.00
detailed_description,1.97
tags,0.00
release_year,0.00


In [9]:
df.dropna(inplace=True)
df.shape

(89421, 5)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89421 entries, 0 to 89617
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  89421 non-null  object
 1   release_date          89421 non-null  object
 2   detailed_description  89421 non-null  object
 3   tags                  89421 non-null  object
 4   release_year          89421 non-null  int32 
dtypes: int32(1), object(4)
memory usage: 3.8+ MB


In [11]:
df = df[df['release_year'] > 2020].reset_index(drop=True)

In [12]:
df['content'] = df['detailed_description'] + ' ' + df['tags']

In [13]:
df = df[['name', 'release_year', 'content']]

In [14]:
df_sample = df.sample(n=1000, random_state=40, ).reset_index(drop=True)
df_sample.shape

(1000, 3)

In [19]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=768c9e6f79ebdca9365c8bbc7cdcbeafcf9fe8d7184c16fda3fea7bb998ad375
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [23]:
from langdetect import detect
df_sample['language'] = df_sample['content'].apply(lambda x: detect(x))
df_sample = df_sample[df_sample['language'] == 'en']

In [24]:
df_sample

,name,release_year,content,language
0,Tim Climpy's Warfair,2024,"A word from Tim Climpy Hi, I'm Tim Climpy and ...",en
1,Crypto Rush,2024,This game is a parody created under the influe...,en
2,You Are Sick,2023,Wishlist Prison Life 2! Join our Discord! Abou...,en
3,A Sinful Camp,2022,◆Featured Systems ◇Full CG mode can be turned ...,en
4,Ticker,2024,This game is a colorful platform game where pl...,en
...,...,...,...,...
995,Drakkon World Builder,2023,Drakkon World Builder is a comprehensive tool ...,en
996,Kaverini Nuuk Adventures,2022,"Welcome to Nuuk, the capital of Greenland! Des...",en
997,Jrago The Demon Hunter,2023,Get ready for an all-new RETRO style gaming ex...,en
998,Insights - Maniac Vortex,2021,Young journalist Thomas wants to write a newsp...,en


In [25]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 981 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          981 non-null    object
 1   release_year  981 non-null    int32 
 2   content       981 non-null    object
 3   language      981 non-null    object
dtypes: int32(1), object(3)
memory usage: 34.5+ KB


# **Preprocessing**

In [26]:
import re

In [27]:
# clean_spcl = re.compile('[/(){}\|@,;]')
# clean_symbol = re.compile('[^0-9a-z #+_]')
# sastrawi = StopWordRemoverFactory()
# stopworda = sastrawi.get_stop_words()
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# def clean_text(text):
#     text = text.lower()
#     text = clean_spcl.sub(' ', text)
#     text = clean_symbol.sub('', text)
#     text = stemmer.stem(text)
#     text = ' '.join(word for word in text.split() if word not in stopworda)
#     return text

In [28]:
import ast
def convert_text(text):
    try:
        teks = ast.literal_eval(text)
        return " ".join([word.lower() for word in teks]) # Convert to lowercase here
    except (ValueError, SyntaxError):
        if isinstance(text, str):
          return " ".join([word.lower() for word in text.split()]) # Handle string cases
        return text

df_sample['content'] = df_sample['content'].apply(convert_text)

<ipython-input-28-103933a3d6f7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['content'] = df_sample['content'].apply(convert_text)


In [29]:
def cleaning_text(text) :
    text = re.sub(r'#\w+', '', text) #remove hashtag
    text = re.sub(r'\b\w*\d\w*\b', '', text) #remove words with numbers in the middle
    text = re.sub(r'@\S+', '', text) #remove mentions (@username)
    text = re.sub(r'https?://\S+|www\.\S+', '', text) #remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) #remove special characters
    return text

df_sample['clean_content'] = df_sample['content'].apply(cleaning_text)

<ipython-input-29-18cfdc3f0d35>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['clean_content'] = df_sample['content'].apply(cleaning_text)


In [30]:
nltk.download('punkt_tab')
df_sample['tokenized'] = df_sample['clean_content'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
<ipython-input-30-8dbcf6368073>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['tokenized'] = df_sample['clean_content'].apply(word_tokenize)


In [31]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
df_sample['stopword'] = df_sample['tokenized'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-31-a4419d1fe065>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['stopword'] = df_sample['tokenized'].apply(lambda x: [word for word in x if word.lower() not in stop_words])


In [32]:
# from stemmer.porter2 import Porter2Stemmer
# stemmer = Porter2Stemmer()
# factory = StemmerFactory()
# porter = PorterStemmer() # untuk stemming kata tertentu dalam bahasa lain
# stemmer = factory.create_stemmer()

# kata_khusus = ['processing']

# kalimat1_stem = []
# for word in kalimat1:
#     if word in kata_khusus:
#         stemmed = porter.stem(word)
#     else:
#         stemmed = stemmer.stem(word)
#     kalimat1_stem.append(stemmed)

# print(kalimat1_stem)

# df_sample['stemming'] = df_sample['stopword'].apply(lambda x: [stemmer.stem(word) for word in x])

In [33]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [41]:
type(df_sample['stopword'].iloc[0])

list

In [42]:
lematizer = WordNetLemmatizer()
df_sample['lematized'] = df_sample['stopword'].apply(lambda x: [lematizer.lemmatize(word) for word in x]).apply(lambda x: " ".join(x))

<ipython-input-42-1c37c3590f1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['lematized'] = df_sample['stopword'].apply(lambda x: [lematizer.lemmatize(word) for word in x]).apply(lambda x: " ".join(x))


# **TF-IDF**

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_sample['lematized'])
tfidf_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [46]:
print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")

TF-IDF Matrix Shape: (981, 13983)


In [47]:
tfidf_similarity

array([[1.        , 0.01704977, 0.01880471, ..., 0.03692024, 0.02030791,
        0.05109753],
       [0.01704977, 1.        , 0.00808075, ..., 0.00488752, 0.01611583,
        0.00970163],
       [0.01880471, 0.00808075, 1.        , ..., 0.01173634, 0.03630862,
        0.03275199],
       ...,
       [0.03692024, 0.00488752, 0.01173634, ..., 1.        , 0.03138387,
        0.03098982],
       [0.02030791, 0.01611583, 0.03630862, ..., 0.03138387, 1.        ,
        0.03645556],
       [0.05109753, 0.00970163, 0.03275199, ..., 0.03098982, 0.03645556,
        1.        ]])

# **Word2Vec**